<a href="https://colab.research.google.com/github/ethanpnguyen/ds4e/blob/main/notebooks/task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## House Prices:

Can you leverage more columns in the dataset for better predictions?

## Initialize

In [34]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

## Load Data

In [35]:
dfMlb = pd.read_csv('/content/house_prices.csv')

In [36]:
dfMlb.dtypes

Suburb            object
Address           object
Rooms              int64
Type              object
Price            float64
Method            object
SellerG           object
Date              object
Distance         float64
Postcode         float64
Bedroom2         float64
Bathroom         float64
Car              float64
Landsize         float64
BuildingArea     float64
YearBuilt        float64
CouncilArea       object
Lattitude        float64
Longtitude       float64
Regionname        object
Propertycount    float64
dtype: object

In [37]:
# Select features and target

X = dfMlb.drop(['Price'], axis=1)
y = dfMlb.loc[:, 'Price']

In [38]:
# Split into training and test sets

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [39]:
# Select numeric features
cols_num = [col for col in X.columns if X[col].dtype in ['int64','float64']]
Xnum = X[cols_num]
Xnum.head()

Xnum_train, Xnum_test, y_train, y_test = train_test_split(Xnum,y, test_size=0.2,random_state=1)

In [40]:
Xnum_train.isna().sum()

Rooms               0
Distance            0
Postcode            0
Bedroom2            0
Bathroom            0
Car                52
Landsize            0
BuildingArea     5193
YearBuilt        4312
Lattitude           0
Longtitude          0
Propertycount       0
dtype: int64

## Build Random Forest Model

In [41]:
from sklearn.ensemble import RandomForestRegressor

def get_random_forest_mae(X_trn,X_tst,y_trn,y_tst):
  mdlRfsMlb = RandomForestRegressor(random_state=1)
  mdlRfsMlb.fit(X_trn, y_trn)
  y_tst_prd = mdlRfsMlb.predict(X_tst)
  mae = mean_absolute_error(y_tst, y_tst_prd)
  return(mae)

In [42]:
## Approach 1: Drop columns with missing values

cols_num_null = [col for col in Xnum.columns if Xnum[col].isna().any()]
Xnum_train_drpnull = Xnum_train.drop(cols_num_null, axis=1)
Xnum_test_drpnull = Xnum_test.drop(cols_num_null, axis=1)

print('MAE from Approach 1 (Drop columns with missing values):')
print(get_random_forest_mae(Xnum_train_drpnull, Xnum_test_drpnull, y_train, y_test))

MAE from Approach 1 (Drop columns with missing values):
176556.1092096132


In [43]:
## Approach 2: Fill missing values with 0

Xnum_train_repnull = Xnum_train.fillna(0)
Xnum_test_repnull = Xnum_test.fillna(0)

print('MAE from Approach 2 (Fill missing values with 0):')
print(get_random_forest_mae(Xnum_train_repnull, Xnum_test_repnull, y_train, y_test))

MAE from Approach 2 (Fill missing values with 0):
167656.98217318885


In [44]:
## Approach 3: Fill missing values with mean value

Xnum_train_repnull = Xnum_train.fillna(Xnum_train.mean())
Xnum_test_repnull = Xnum_test.fillna(Xnum_train.mean())

print('MAE from Approach 3 (Fill missing values with mean):')
print(get_random_forest_mae(Xnum_train_repnull, Xnum_test_repnull, y_train, y_test))

MAE from Approach 3 (Fill missing values with mean):
166170.5766405428


In [45]:
# Going forward, let us replace all missing numeric values with the column mean

X_train[cols_num] = Xnum_train_repnull[cols_num]
X_test[cols_num] = Xnum_test_repnull[cols_num]

## Non-numerical Features

In [49]:
cols_cat = [col for col in X.columns if X[col].dtype in ['object'] and X[col].nunique()<10]

In [50]:
from sklearn import preprocessing

Xle_train = X_train.copy()
Xle_test = X_test.copy()

le = preprocessing.LabelEncoder()

for col in cols_cat:
  Xle_train[col] = le.fit_transform(Xle_train[col])
  Xle_test[col] = le.transform(Xle_test[col])

In [51]:
print('MAE from Label Encoding all Categorical columns:')
print(get_random_forest_mae(Xle_train[cols_num+cols_cat],Xle_test[cols_num+cols_cat],y_train,y_test))

MAE from Label Encoding all Categorical columns:
156274.91509818358


## Build Gradient Boosting Model

In [55]:
from xgboost import XGBRegressor

mdlXgbMlb = XGBRegressor()
mdlXgbMlb.fit(Xle_train[cols_num+cols_cat],y_train)
y_test_pred = mdlXgbMlb.predict(Xle_test[cols_num+cols_cat])
mae = mean_absolute_error(y_test,y_test_pred)

print('MAE from default XGBoost model:')
print(mae)

[18:02:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MAE from default XGBoost model:
183775.97442240428


In [57]:
# Build and score a tuned Gradient Boosting Model

mdlXgbMlb = XGBRegressor(n_estimators=5000,learning_rate=0.01,max_depth=5)
mdlXgbMlb.fit(Xle_train[cols_num+cols_cat],y_train)
y_test_pred = mdlXgbMlb.predict(Xle_test[cols_num+cols_cat])
mae = mean_absolute_error(y_test,y_test_pred)

print('MAE from default XGBoost model:')
print(mae)

[18:04:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MAE from default XGBoost model:
150992.05709223123
